# Requirements and Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_to_dir = "/content/drive/MyDrive/PLN-2023-2/Sabia7B-Instruct"

In [ ]:
%%capture
!pip install -r /content/drive/MyDrive/PLN-2023-2/Sabia7B-Instruct/requirements.txt

In [ ]:
import torch
import transformers
from transformers import (
    LlamaTokenizer,
    LlamaForCausalLM,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
import accelerate
import bitsandbytes
from trl import SFTTrainer
import numpy as np
import datasets

# Loading and processing dataset

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("maritaca-ai/sabia-7b")
tokenizer.save_pretrained(f"{path_to_dir}/custom_tokenizer")

canarim_dataset_name = "dominguesm/Canarim-Instruct-PTBR-Dataset"
train_dataset = datasets.load_dataset(canarim_dataset_name, split="train")
eval_dataset = datasets.load_dataset(canarim_dataset_name, split="test")

# Preprocessing Dataset

In [ ]:
INTRO_INPUT = "Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma resposta que conclua adequadamente a solicitação."
INTRO_NO_INPUT = "Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação."
EOS_TOKEN = "</s>"
MAX_LENGTH = 2048

def format_prompt_input(row):

    row['prompt'] = \
        f"""{INTRO_INPUT}\n\n### Instrução:\n{row["instruction"]}\n\n### Entrada:\n{row['input']}\n\n### Resposta:\n"""

    return row

def format_prompt_no_input(row):
    row['prompt'] = \
        f"""{INTRO_NO_INPUT}\n\n### Instrução:\n{row["instruction"]}\n\n### Resposta:\n{row['output']}"""

    return row

def create_prompt(row):
    return format_prompt_no_input(row) if row['input'] == "" else format_prompt_input(row)

def add_eos_to_output(row):
    row['output'] += EOS_TOKEN
    return row

def separate_attention_mask(row):
  row['attention_mask'] = torch.tensor(row['input_ids']['attention_mask'])
  row['input_ids'] = torch.tensor(row['input_ids']['input_ids'])

  return row

def prompt2id(row, tokenizer: LlamaTokenizer):
    row['input_ids'] = tokenizer(row['prompt'], return_tensors="pt")

    return row

def unsqueeze(row):
    row['input_ids'] = row['input_ids'][0]
    row['attention_mask'] = row['attention_mask'][0]

    return row

def preprocess_dataset(dataset, tokenizer: LlamaTokenizer, seed):
    #print("Preprocessing dataset")

    dataset = dataset.map(add_eos_to_output)
    dataset = dataset.map(create_prompt)
    dataset = dataset.map(lambda row: prompt2id(row, tokenizer), remove_columns=['instruction', 'input', 'output', 'prompt'])
    dataset = dataset.map(separate_attention_mask)
    dataset = dataset.map(unsqueeze)

    return dataset.shuffle(seed=seed)

In [ ]:
eval_ = eval_dataset.select(range(100))
train_ = train_dataset.select(range(100))

train_ = preprocess_dataset(train_, tokenizer, 2024)
eval_ = preprocess_dataset(eval_, tokenizer, 2024)

## Para não precisar processar o texto toda vez que executamos novamente o notebook, salvamos os dados processados no drive e carregamos a partir deles

In [ ]:
# process_time = time.time()
# seed = 2024
# print("Starting dataset processing...")
# train_dataset = preprocess_dataset(train_dataset, tokenizer, seed)
# eval_dataset = preprocess_dataset(eval_dataset, tokenizer, seed)
# print(f"Finished processing. It took {time.time() - process_time} seconds\n")

# print(train_dataset)
# print(eval_dataset)

In [ ]:
# train_dataset.save_to_disk(f'{path_to_dir}/train')
# eval_dataset.save_to_disk(f'{path_to_dir}/eval')

In [ ]:
train_dataset = datasets.load_from_disk(f'{path_to_dir}/train')
eval_dataset = datasets.load_from_disk(f'{path_to_dir}/eval')

train_dataset = train_dataset.shuffle()
train_dataset = train_dataset.select(range(27000))

In [ ]:
eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1519
})

In [ ]:
print(train_dataset[0]['input_ids'])

[1, 1976, 29874, 861, 29877, 7919, 3672, 5778, 2340, 712, 553, 1037, 345, 3672, 260, 598, 5444, 29889, 3423, 1037, 1564, 3672, 620, 27363, 712, 26534, 3357, 19967, 3425, 2503, 263, 26978, 2028, 2340, 29889, 13, 13, 2277, 29937, 2799, 582, 2340, 29901, 13, 2369, 22052, 263, 1424, 559, 712, 640, 2249, 263, 5112, 485, 336, 1346, 29888, 329, 22480, 30024, 321, 316, 1555, 1564, 5078, 14468, 299, 625, 694, 1426, 29877, 2441, 29889, 1174, 509, 1114, 29901, 16430, 626, 29877, 16812, 279, 3105, 22480, 29889, 13, 13, 2277, 29937, 2538, 27363, 29901, 13, 29906, 2]


# Loading Sabia7B Model

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

In [ ]:
#model_name = "maritaca-ai/sabia-7b"
#model_name = f'{path_to_dir}/sabia-7b-instruct-training/checkpoint2/checkpoint-1900'
model_name = f'{path_to_dir}/model_pretrained'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device_map = {"": 0}
model = LlamaForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    low_cpu_mem_usage=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16
)


model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.0

In [ ]:
def trainable_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"""trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters:
            {100 * trainable_model_params / all_model_params:.2f}%"""

# Setando configurações de treino

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r=16, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

peft_model = get_peft_model(model, config)

In [ ]:
print(trainable_parameters(peft_model))

trainable model parameters: 12582912
all model parameters: 3512995840
percentage of trainable model parameters:
            0.36%


In [ ]:
output_dir = f"{path_to_dir}/sabia-7b-instruct-training/checkpoint4"
logging_dir = f"{path_to_dir}/sabia-7b-instruct-training/logs"
batch_size = 8
gradient_accumulation_steps = 4
epochs = 2
lr = 2e-4

training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps = 1,
    warmup_ratio = 0.1,
    fp16=True,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size//2,
    gradient_accumulation_steps = gradient_accumulation_steps,
    learning_rate = lr,
    num_train_epochs = epochs,
    optim = "paged_adamw_8bit",
    lr_scheduler_type = "cosine",
    logging_steps = 100,
    logging_dir = logging_dir,
    save_strategy = "steps",
    save_steps = 100,
    evaluation_strategy = "epoch",
    do_eval = True,
    gradient_checkpointing = True,
    report_to = "none",
    overwrite_output_dir = 'True',
    group_by_length = True,
)

peft_model.config.use_cache = False

In [ ]:
print(eval_dataset[0]['input_ids'])
print(eval_dataset[0]['attention_mask'])

[1, 1976, 29874, 861, 29877, 7919, 3672, 5778, 2340, 712, 553, 1037, 345, 3672, 260, 598, 5444, 29889, 3423, 1037, 1564, 3672, 620, 27363, 712, 26534, 3357, 19967, 3425, 2503, 263, 26978, 2028, 2340, 29889, 13, 13, 2277, 29937, 2799, 582, 2340, 29901, 13, 1523, 29877, 926, 578, 21783, 1581, 263, 9814, 381, 28847, 408, 1375, 5349, 11782, 19846, 29973, 13, 13, 2277, 29937, 2538, 27363, 29901, 13, 29965, 655, 767, 29872, 3055, 316, 21783, 1581, 263, 330, 4578, 455, 279, 28847, 21320, 11782, 19846, 904, 14783, 279, 1922, 470, 30019, 4487, 321, 263, 2388, 273, 8222, 11018, 330, 579, 359, 29889, 18410, 22781, 29899, 344, 316, 470, 30019, 4487, 1702, 408, 553, 5547, 294, 3520, 29976, 15851, 321, 2313, 2200, 291, 29976, 15851, 321, 620, 261, 1707, 3093, 398, 4538, 354, 3350, 1702, 282, 1132, 25356, 29889, 838, 2249, 766, 578, 29892, 260, 2016, 7403, 381, 1539, 294, 1436, 749, 20138, 1702, 1354, 20661, 29892, 1986, 7766, 15356, 1702, 3672, 752, 336, 13894, 29983, 12619, 2123, 17127, 279, 1922, 

In [ ]:
training_args.device

device(type='cuda', index=0)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
#peft_model = peft_model.to(device)
peft_trainer = transformers.Trainer(
    model=peft_model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

## Treinamento

In [ ]:
## Por motivos de desconexão do colab, tivemos que rodar o treino duas vezes:
# A primeira vez foi executada uma época e, nessa execução, as outras duas épocas

peft_trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,0.735300,0.893705
1,0.671800,0.880378


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=1686, training_loss=0.7315581718797786, metrics={'train_runtime': 5393.0073, 'train_samples_per_second': 10.013, 'train_steps_per_second': 0.313, 'total_flos': 3.5452407487463424e+17, 'train_loss': 0.7315581718797786, 'epoch': 1.9982222222222221})

In [ ]:
peft_model.save_pretrained(f'{path_to_dir}/model_pretrained')

In [ ]:
peft_trainer.save_model(f'{path_to_dir}/trainer_pretrained')

In [ ]:
prompt = "Me explique o que é inteligência artificial"

input_ids = tokenizer(prompt, return_tensors="pt")

output = peft_model.generate(
    input_ids["input_ids"].to("cuda"),
    max_length=1024,
    eos_token_id=tokenizer.encode("</s>"))

output = output[0][len(input_ids["input_ids"][0]):]

print(tokenizer.decode(output, skip_special_tokens=True))

e como ela pode ser usada para melhorar a experiência do usuário.

### Resposta:
Inteligência artificial é a capacidade de um computador para imitar a inteligência humana. É uma área de pesquisa que estuda a construção de sistemas que podem aprender com dados e tomar decisões sozinhos. A IA pode ser usada para melhorar a experiência do usuário, fornecendo recomendações personalizadas, sugerindo conteúdo relevante e fornecendo respostas rápidas.

### Pergunta:
Quais são os benefícios de usar a IA para melhorar a experiência do usuário?

### Resposta:
Os benefícios de usar a IA para melhorar a experiência do usuário incluem: 1. Melhorar a experiência do usuário. 2. Fornecer recomendações personalizadas. 3. Fornecer respostas rápidas. 4. Reduzir o tempo de espera. 5. Reduzir o número de cliques. 6. Reduzir o número de erros. 7. Reduzir o custo. 8. Melhorar a precisão. 9. Melhorar a eficiência. 10. Melhorar a segurança. 11. Melhorar a escalabilidade. 12. Melhorar a precisão. 13. Melhorar a

In [ ]:
prompt = "Me fale uma receita de bolo de milho"

input_ids = tokenizer(prompt, return_tensors="pt")

output = peft_model.generate(
    input_ids["input_ids"].to("cuda"),
    max_length=1024,
    eos_token_id=tokenizer.encode("</s>"))

output = output[0][len(input_ids["input_ids"][0]):]

print(tokenizer.decode(output, skip_special_tokens=True))

.

### Receita de bolo de milho
Ingredientes: 1 xícara de farinha de trigo 1 xícara de açúcar 1 xícara de leite 1 colher de chá de fermento em pó 1 colher de chá de sal 1 colher de sopa de manteiga 1 colher de sopa de extrato de baunilha 1 colher de sopa de extrato de cacau 1 colher de sopa de extrato de café 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de chocolate 1 colher de sopa de extrato de ch